# Final Code

<div class="alert alert-success">
All of the concepts on choosing the subsets and data particularities in this code is concluded from the 'EDA - analysing Nans' notebook, which is also included in this repository. <br> <br>

</div>


# Dataset tuning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Math, Latex
from scipy import stats, linalg
% matplotlib inline


df = pd.read_csv('SPcities2.csv', decimal=',', encoding='Iso-8859-1', sep=';')


df = df[df.year > 2001]
df = df[df.year < 2016]
df.head()

,city,year,gdp_per,gdp_growth,private_inv,pub_inv,export,import,violence,HDI,...,hosp_rooms,hosp_rooms_per,jobs,jobs_revenue,population,urban_pop,rural_pop,olding,urbing,fundamental
2,Adamantina,2002,8355.50,NaN,NaN,NaN,NaN,NaN,326,NaN,...,NaN,NaN,6006.0,654.70,33636,30635,3001.0,67.36,91.08,591.0
3,Adamantina,2003,9936.93,0.190718,NaN,NaN,7509556.0,NaN,289,NaN,...,NaN,NaN,6223.0,709.64,33677,30796,2881.0,70.87,91.45,677.0
4,Adamantina,2004,9983.64,0.005835,NaN,NaN,6864268.0,NaN,433,NaN,...,NaN,NaN,6751.0,762.66,33715,30958,2757.0,74.62,91.82,594.0
5,Adamantina,2005,11264.43,0.129928,NaN,NaN,42493060.0,1218682.0,395,NaN,...,NaN,NaN,6806.0,769.85,33764,31120,2644.0,78.58,92.17,474.0
6,Adamantina,2006,13247.35,0.176731,NaN,NaN,81851232.0,3702790.0,318,NaN,...,NaN,NaN,7323.0,893.81,33784,31283,2501.0,82.71,92.60,NaN


In [2]:
data = df[df.year == 2013] # like we saw in the EDA on missing values, 2013 should be the best year for our analysis
                           # this was the case because, for this method, we need the year with more information, in this case 2013

In [3]:
# checking the distribution of non NaN values in the features
notnull = data[data.notnull()].count()
notnull

city               645
year               645
gdp_per            645
gdp_growth         645
private_inv         73
pub_inv            409
export             354
import             377
violence           645
HDI                  0
educ_superior        0
primary_enrolls    645
density_pop        645
value_add          645
agriculture_add    644
industry_add       645
services_add       645
eletricity         645
tax_revenue          0
hosp_rooms         358
hosp_rooms_per     358
jobs               645
jobs_revenue       645
population         645
urban_pop          645
rural_pop          615
olding             645
urbing             645
fundamental        645
dtype: int64

Here we can see that we need to drop 3 features here, since there is no data : HDI, educ_superior and tax_revenue

In [4]:
# cleaning the all null features

data_clean = data
data_clean = data_clean.drop('HDI', 1)
data_clean = data_clean.drop('educ_superior', 1)
data_clean = data_clean.drop('tax_revenue', 1)

data_clean.head()

,city,year,gdp_per,gdp_growth,private_inv,pub_inv,export,import,violence,primary_enrolls,...,hosp_rooms,hosp_rooms_per,jobs,jobs_revenue,population,urban_pop,rural_pop,olding,urbing,fundamental
13,Adamantina,2013,24097.48,0.096675,NaN,696535.15,29152328.0,239286.0,301,101.16,...,284.0,8.39,8936.0,1600.40,33845,32233,1612.0,113.53,95.24,476.0
30,Adolfo,2013,20871.22,0.382122,2.4,408000.00,NaN,3500.0,48,107.65,...,NaN,NaN,816.0,1625.53,3523,3206,317.0,97.44,91.00,39.0
47,Aguaí,2013,19918.32,0.163114,NaN,140850.00,NaN,311038.0,495,97.65,...,NaN,NaN,6289.0,1634.36,33179,30148,3031.0,58.22,90.86,242.0
64,Águas da Prata,2013,14559.93,0.101242,NaN,NaN,NaN,NaN,34,84.21,...,NaN,NaN,956.0,1674.54,7652,6907,745.0,114.88,90.26,46.0
81,Águas de Lindóia,2013,19977.22,0.258870,NaN,619445.00,178037.0,1966779.0,257,105.52,...,42.0,2.39,4663.0,1468.63,17610,17452,158.0,82.49,99.10,121.0


------------------------------------------------ END OF DATASET TUNING ----------------------------------------------

The I-Distance method:

<img src='idistance.png'> </img>

<img src='disceffect.png'> </img>

"where di(r, s) is the distance between the values of variable Xi for er and es e.g. the discriminate effect" and <br> <br>

"σi the standard deviation of Xi, and rji.12 … j − 1 is a partial coefficient of the correlation between Xi and Xj, (j < i)"

# Creating the tools to use the method

In [5]:
# this idea for partial correlation was obtained from : https://gist.github.com/fabianp/9396204419c7b638d38f

## Partial Correlation Matrix

from scipy import stats, linalg

def partial_corr(C):
    """
    Returns the sample linear partial correlation coefficients between pairs of variables in C, controlling 
    for the remaining variables in C.
    Parameters
    ----------
    C : array-like, shape (n, p)
        Array with the different variables. Each column of C is taken as a variable
    Returns
    -------
    P : array-like, shape (p, p)
        P[i, j] contains the partial correlation of C[:, i] and C[:, j] controlling
        for the remaining variables in C.
    """
    
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]

            res_j = C[:, j] - C[:, idx].dot( beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)
            
            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr

        
    return P_corr

In [6]:
# The reference hyperplane to calculate all the distances from.
# Here we are going to set the city with the smallest value for GDP per capita

data_clean.sort_values('gdp_per', ascending=True).head()

,city,year,gdp_per,gdp_growth,private_inv,pub_inv,export,import,violence,primary_enrolls,...,hosp_rooms,hosp_rooms_per,jobs,jobs_revenue,population,urban_pop,rural_pop,olding,urbing,fundamental
4450,Itapirapuã Paulista,2013,6956.90,0.098176,NaN,917044.27,NaN,NaN,12,102.25,...,NaN,NaN,521.0,1266.38,3954,1959,1995.0,42.32,49.54,64.0
10196,Taquaral,2013,7307.11,-0.110965,NaN,91200.00,NaN,NaN,9,89.24,...,NaN,NaN,500.0,1159.92,2729,2623,106.0,74.68,96.12,34.0
8462,Riversul,2013,7386.98,0.132233,NaN,NaN,NaN,NaN,72,94.24,...,NaN,NaN,533.0,1188.74,6003,4423,1580.0,76.79,73.68,64.0
7816,Pracinha,2013,7650.01,0.210545,NaN,NaN,NaN,NaN,7,99.51,...,NaN,NaN,264.0,1307.56,2864,1372,1492.0,77.06,47.91,17.0
285,Álvaro de Carvalho,2013,7653.97,-0.031318,NaN,1162680.60,NaN,NaN,29,100.62,...,NaN,NaN,359.0,1443.34,4763,3081,1682.0,60.52,64.69,55.0


Here, since the city of Itapirapua Paulista has the smallest value for the features of our most interest, it will be the hyperplane reference for our calculations.

In [7]:
data_values = data_clean.drop('year' ,1).fillna(0) # Base dataset for all calculations


reference = data_values.sort_values('gdp_per', ascending=True).iloc[0:1,:] # Base reference for all calculations

# Building the function to calculate the I-distance

Like the steps followed by <a href="http://www.sciencedirect.com/science/article/pii/S0264999314000558">reference article </a> , we should : <br>

<ol>
<li>Calculate the value of the discriminate effect of the variable X1, the most significant variable, that which provides the largest amount of information on the phenomena that is to be ranked; </li>
<li> Add the value of the discriminate effect of X2 which is not covered by X1;</li>
<li> Add the value of the discriminate effect of X3 which is not covered by X1 and X2;</li>
<li> Repeat the procedure for all variables;</li>
<li> Calculte the correlation coefficient of each indicator with the I-distance;</li>
<li> Exclude the smallest non significant coefficient;</li>
<li> Repeat (6) until only significant features remain.</li>


In [8]:
## Creating a function to the calculation to operate 1-4 above: 

def I_distance(data_values, reference):
    
    #getting the correlation matrix done
    data_corr = data_values.drop('city' ,1).fillna(0)
    col_labels = list(data_corr.columns)
    corre = pd.DataFrame(partial_corr(data_corr), columns=col_labels)
                         
    #starting the I-Distance calculation                     
    ncolums = data_corr.shape[1]
    nrows = data_corr.shape[0]
    somatory = []
    i2_index = {'city': [], 'I-Distance': []}
    for r in range(0,nrows):
        somatory = []    
        for i in range(0,ncolums):
            prod = 1
            summ_1 = 0
            summ_1 = ((data_corr.iloc[r,i] - reference.iloc[0,i+1])**2)/np.var(data_corr.iloc[:,i]) #distance squared divided by variance
            for k in range(i+1):
                if corre.iloc[i,k] == 1:
                    prod *= 1
                else:
                    prod *= (1-(corre.iloc[i,k])**2)
            somatory.append(summ_1*prod)
        i2_index['city'].append(data_values.iloc[r,0])
        i2_index['I-Distance'].append(np.cumsum(somatory)[-1])
    

## 5 - 6 selecting sigficant features

# calculating the rank of features and p-values

    distance = pd.DataFrame(i2_index)

    data = data_values.iloc[:,1:]
    feature_rank = {'feature': [], 'r': [], 'p-value': []}

    # creating the rank with correlation, p-values and name of features
    for col, values in data.iteritems():
        feature_rank['feature'].append(col)
        feature_rank['r'].append(stats.pearsonr(values, distance['I-Distance'])[0])
        feature_rank['p-value'].append(stats.pearsonr(values, distance['I-Distance'])[1])
    

    # manipulating the rank to select smallest correlation value with no significant p-value valor (>0.01)
    rank = pd.DataFrame(feature_rank)
    lenght_rank = len(list(rank.columns))
  

    for x in range(lenght_rank):
        excluded = {'feature' : "" , 'p-value' : 0 }
        chk = 0
        if rank.sort_values('r').iloc[x,1] >= 0.05 and x <= lenght_rank:  # check if the smallest r value is significant
            excluded['feature'] = rank.sort_values('r').iloc[x,0]
            excluded['p-value'] = rank.sort_values('r').iloc[x,1]
            chk = 1
            break
        elif rank.sort_values('r').iloc[x,1] <= 0.05 and x == lenght_rank:
            chk = 0
            break
        else:
            chk = 0
            continue

    if chk == 1: # returning new data frame if there is some variable to exclude, otherwise is done
        n_values = data_values.drop(excluded['feature'], 1)
        n_ref = reference.drop(excluded['feature'], 1)
        return 'loop',distance,  n_values, n_ref
    else:
        return 'done', distance, rank, 'ref' # if no features to drop, return distance and rank
        


In [9]:
# 7 - Iterating for the final model until we gett only significant features:

def Final_distance(data_values, reference):
    check, distance, rank, ref = I_distance(data_values, reference)
    
    while True:
        if check == 'loop':
            check, distance, rank, ref = I_distance(rank, ref)
            continue
        else:
            break
    return check, distance, rank

In [10]:
check1, distance1, rank1 = Final_distance(data_values, reference)

In [11]:
rank1.sort_values('r', ascending=False)

,feature,p-value,r
4,violence,0.000000e+00,0.990580
11,jobs,0.000000e+00,0.989353
8,services_add,0.000000e+00,0.985795
6,value_add,0.000000e+00,0.982313
14,urban_pop,0.000000e+00,0.978691
13,population,0.000000e+00,0.978607
10,hosp_rooms,0.000000e+00,0.976214
16,fundamental,0.000000e+00,0.972127
1,pub_inv,0.000000e+00,0.951517
9,eletricity,8.198462e-314,0.944806


As we can see, of the features of our interest, public investment seems to be a important one, once it 'survived'and has as relevant participation on the index calculation. Otherwise, private investment seems to be irrelevant for economic growth in this particular case. <br><br>

Now, let's make some separate experiments and, using the same year of 2013, try to isolate our 2 variables of interest, public and private investment, and see how they perform without each others presence only in the city where they have not null entries. <b>This seeks to answer wheter, in the cities where they are important, what are their relevance in the I-Distance model. 

<h4>Public Investment: </h4>

In [41]:
# let's prepare our dataset 

public = data_clean[data_clean.pub_inv.notnull()].drop(['year', 'private_inv'], 1).fillna(0) #this will be our dataset for this part

public.head(5)

,city,gdp_per,gdp_growth,pub_inv,export,import,violence,primary_enrolls,density_pop,value_add,...,hosp_rooms,hosp_rooms_per,jobs,jobs_revenue,population,urban_pop,rural_pop,olding,urbing,fundamental
13,Adamantina,24097.48,0.096675,696535.15,29152328.0,239286.0,301,101.16,82.27,750130.73,...,284.0,8.39,8936.0,1600.40,33845,32233,1612.0,113.53,95.24,476.0
30,Adolfo,20871.22,0.382122,408000.00,0.0,3500.0,48,107.65,16.69,71968.31,...,0.0,0.00,816.0,1625.53,3523,3206,317.0,97.44,91.00,39.0
47,Aguaí,19918.32,0.163114,140850.00,0.0,311038.0,495,97.65,69.89,611163.92,...,0.0,0.00,6289.0,1634.36,33179,30148,3031.0,58.22,90.86,242.0
81,Águas de Lindóia,19977.22,0.258870,619445.00,178037.0,1966779.0,257,105.52,292.87,329839.01,...,42.0,2.39,4663.0,1468.63,17610,17452,158.0,82.49,99.10,121.0
115,Águas de São Pedro,34159.91,0.182283,659000.00,0.0,69149.0,53,217.62,511.55,92412.85,...,4.0,1.41,1366.0,1660.75,2834,2834,0.0,169.62,100.00,45.0


In [42]:
# Let`s get our reference, being the minimum value for GDP per capita, as we did above

pub_ref = public.sort_values('gdp_per', ascending=True).iloc[0:1,:]
pub_ref # it will be the same city, but we now have our reference

,city,gdp_per,gdp_growth,pub_inv,export,import,violence,primary_enrolls,density_pop,value_add,...,hosp_rooms,hosp_rooms_per,jobs,jobs_revenue,population,urban_pop,rural_pop,olding,urbing,fundamental
4450,Itapirapuã Paulista,6956.9,0.098176,917044.27,0.0,0.0,12,102.25,9.73,26929.78,...,0.0,0.0,521.0,1266.38,3954,1959,1995.0,42.32,49.54,64.0


In [43]:
checkpub1, distancepub1, rankpub1 = Final_distance(public, pub_ref)

In [45]:
rankpub1.sort_values('r', ascending=False)

,feature,p-value,r
4,violence,0.000000e+00,0.991080
11,jobs,0.000000e+00,0.990186
8,services_add,0.000000e+00,0.989622
6,value_add,2.775463e-319,0.986085
14,urban_pop,7.514347e-299,0.982434
13,population,2.604918e-298,0.982325
10,hosp_rooms,7.664060e-280,0.978172
16,fundamental,1.889556e-279,0.978074
9,eletricity,1.085011e-246,0.968076
1,pub_inv,1.044462e-211,0.952191


It looks like the situation from our first run remains the same, so public investment is still relevant and in the same level, in the cities where it appears in the year of 2013. <br><br>

Let's take a look on the private investment:

<b> Private Investment

In [46]:
# let's prepare our dataset 

private = data_clean[data_clean.private_inv.notnull()].drop(['year', 'pub_inv'], 1).fillna(0) #this will be our dataset for this part

private.head(5)

,city,gdp_per,gdp_growth,private_inv,export,import,violence,primary_enrolls,density_pop,value_add,...,hosp_rooms,hosp_rooms_per,jobs,jobs_revenue,population,urban_pop,rural_pop,olding,urbing,fundamental
30,Adolfo,20871.22,0.382122,2.40,0.000000e+00,3500.0,48,107.65,16.69,71968.31,...,0.0,0.00,816.0,1625.53,3523,3206,317.0,97.44,91.00,39.0
319,Americana,46270.61,0.151524,163.08,2.212849e+08,642267114.0,4599,107.71,1627.42,8453589.31,...,683.0,3.13,82334.0,2127.00,217960,216942,1018.0,79.10,99.53,2777.0
472,Aparecida,21598.81,0.140462,11.00,0.000000e+00,775806.0,1430,109.55,290.87,714664.67,...,84.0,2.39,10456.0,1543.75,35219,34707,512.0,67.45,98.55,420.0
574,Araraquara,35017.73,0.087044,64.00,1.786423e+09,102713236.0,4097,104.88,214.29,6613773.71,...,586.0,2.72,75373.0,2096.92,215080,208966,6114.0,85.24,97.16,2291.0
591,Araras,34363.44,0.043158,0.70,2.711129e+08,60910196.0,1928,101.00,190.06,3675866.05,...,557.0,4.54,38735.0,2055.27,122554,116185,6369.0,73.21,94.80,1380.0


In [48]:
# Let`s get our reference, being the minimum value for GDP per capita, as we did above

priv_ref = private.sort_values('gdp_per', ascending=True).iloc[0:1,:]
priv_ref # this is a different city from the other 2 situations

,city,gdp_per,gdp_growth,private_inv,export,import,violence,primary_enrolls,density_pop,value_add,...,hosp_rooms,hosp_rooms_per,jobs,jobs_revenue,population,urban_pop,rural_pop,olding,urbing,fundamental
1645,Cachoeira Paulista,14903.74,0.162825,11.0,64032.0,160226.0,387,103.65,106.8,435614.15,...,47.0,1.53,5571.0,1584.19,30756,25289,5467.0,67.43,82.22,341.0


In [49]:
checkpri1, distancepri1, rankpri1 = Final_distance(private, priv_ref)

In [50]:
rankpri1.sort_values('r', ascending=False)

,feature,p-value,r
12,jobs,1.037648e-30,0.920596
7,value_add,6.551187e-28,0.903999
3,import,3.315790e-26,0.892145
9,services_add,2.818468e-25,0.885036
4,violence,5.071062e-23,0.865625
15,urban_pop,3.131201e-19,0.824532
14,population,4.680388e-19,0.822333
8,industry_add,1.016742e-18,0.818001
10,eletricity,5.404623e-18,0.808266
17,fundamental,5.166454e-17,0.794170


<div class="alert alert-success">
Here we could see that 2013 should be the better year to apply the method since we have the most of our main predictors. <br> <br>

Another thing is that, beginning from 2010 and going to 2013, could be a good period if we choose to use the mean of more than 1 years to calcule the hyperplanes, since we have important data concetrated in this years.  <br>
</div>